In [1]:
import ipywidgets
from ipyleaflet import (
    Map,
    Marker,
    TileLayer,
    ImageOverlay,
    Polyline,
    Polygon,
    Rectangle,
    Circle,
    CircleMarker,
    GeoJSON,
    GeomanDrawControl,
)

from traitlets import link

In [2]:
import json

with open("simple.geo.json") as f:
    data = json.load(f)

In [3]:
for i, f in enumerate(data["features"]):
    f["id"] = i

In [4]:
d1 = data.copy()
d2 = data.copy()

d1["features"] = d1["features"][:1]
d2["features"] = d2["features"][1:]


In [5]:
center = [46.475212657477016, 6.3198722284199675]
zoom = 9
m = Map(center=center, zoom=zoom, layout=dict(height="600px"))

g1 = GeoJSON(data=d1, pm_ignore=True, snap_ignore=True,     
            style={
        'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.1, 'weight': 1
    },
           )

g2 = GeoJSON(data=d2, pm_ignore=True, snap_ignore=False)
m.add(g1)
m.add(g2)

draw_data = data["features"][0]
draw_data["properties"]["type"] = "circlemarker"
dc = GeomanDrawControl(
    marker={},
    circlemarker={},
    polygon={},
    data=[draw_data],
    # hover_style={"color": "red", "dashArray": "0", "fillOpacity": 1, "weight": 8.0},
    limit_markers_to_count=5,
    continuous_update=False,
    custom_controls=[{
        "name": "save",
        "title": "Save",
        "className": "control-icon leaflet-pm-icon-delete",
        "event": "save",
        "block": "custom"
    }]
)

def handle_draw(target, action, geo_json):
    print(action)
    print(geo_json)

def on_value_change(change):
    print("DATA CHANGE")
    print(change)

def on_custom_button(*args, **kwargs):
    print(args)
    print(kwargs)
    dc.sync_data()

dc.observe(on_value_change, names='data')
dc.on_draw(handle_draw)

dc.on_click_custom_control(on_custom_button)
m.add(dc)
m

Map(center=[46.475212657477016, 6.3198722284199675], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [6]:
# Explicit data sync
dc.sync_data()

In [7]:
import io
from collections.abc import Callable
from typing import cast

import ipyleaflet
import ipywidgets
import matplotlib as mpl
import reacton.ipywidgets as w
import solara
from ipyleaflet import WidgetControl
from matplotlib import pyplot as plt
from traitlets import default, traitlets
from ipyleaflet import basemaps

<IPython.core.display.Javascript object>

In [8]:

@solara.component
def MyGeomanDrawControl(
    *,
    on_draw=None,
    data=None,
    position="topright",
    current_mode=None,
    polygon=None,
    circle=None,
    marker=None,
    circlemarker=None,
    rectangle=None,
    polyline=None,
    text=None,
    edit=True,
    remove=True,
    drag=True,
    cut=True,
    rotate=True,
    hide_controls=False,
    limit_markers_to_count=-1,
    continuous_updates=True,
):
    """
    Args:
        on_draw:
        data:
        position:
        current_mode:
        polygon:
        circle:
        marker:
        circlemarker:
        rectangle:
        polyline:
        text:
        edit:
        remove:
        drag:
        cut:
        rotate:
        hide_controls:
        limit_markers_to_count:

    Returns:

    """
    if polyline is None:
        polyline = {}
    if rectangle is None:
        rectangle = {}
    if circlemarker is None:
        circlemarker = {}
    if marker is None:
        marker = {}
    if circle is None:
        circle = {}
    if polygon is None:
        polygon = {}
    if text is None:
        text = {}
    if data is None:
        data = []

    draw_control = ipyleaflet.GeomanDrawControl.element(
        data=data,
        # explicitly set these to empty dict to disable them
        circle=circle,
        marker=marker,
        circlemarker=circlemarker,
        polygon=polygon,
        rectangle=rectangle,
        polyline=polyline,
        text=text,
        position=position,
        edit=edit,
        remove=remove,
        current_mode=current_mode,
        drag=drag,
        cut=cut,
        rotate=rotate,
        hide_controls=hide_controls,
        limit_markers_to_count=limit_markers_to_count,
        continuous_updates=continuous_updates,
    )

    def add_event_handler():
        if on_draw is None:
            return

        widget = cast(ipyleaflet.GeomanDrawControl, solara.get_widget(draw_control))
        if isinstance(on_draw, Callable):
            widget.on_draw(on_draw)
        else:
            for callback in on_draw:
                widget.on_draw(callback)

        # def log_data(change):
        #     log.warning("data receved from geoman draw control: ", change=change)
        #
        # widget.observe(log_data, names=["data"])

        def cleanup():
            widget.on_draw(on_draw, remove=True)

        return cleanup

    solara.use_effect(add_event_handler, dependencies=[])

    return draw_control


In [9]:
@solara.component
def MyMap(
    *,
    layers,
    controls=(),
    center=(5,10),
    zoom=10,
    scroll_wheel_zoom=True,
    layout=None,
    on_zoom=None,
    on_center=None,
    zoom_control=True,
    **kwargs,
):
    return ipyleaflet.Map.element(
        layout=layout or w.Layout(width="100%", height="400px", margin="0"),
        layers=layers,
        center=center,
        on_center=on_center,
        zoom=zoom,
        on_zoom=on_zoom,
        controls=controls,
        zoom_control=zoom_control,
        scroll_wheel_zoom=scroll_wheel_zoom,
        **kwargs,
    ).meta(ref="map-widget")

In [20]:
controls = [MyGeomanDrawControl(
    position="topleft",
    marker={},
    circlemarker={},
    polygon={"pathOptions": {"stroke": True, "color": "#3388ff", "weight": 3}},
                              )]

MyMap(
    layers=[ipyleaflet.TileLayer.element(
        url=basemaps.OpenStreetMap.Mapnik.url)
           ],
    controls=controls,
)

Cannot show ipywidgets in text